<a href="https://colab.research.google.com/github/beloveddie/AI-Craft/blob/main/retrieval_augmented_ai_agent_fakestore_api_usecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.8 MB/s eta 0:00:00


In [3]:
# set OPENAI_API_KEY from Colab secrets
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY_SOLID')

In [6]:
import requests
from typing import Dict, List, Optional, Any, Union
from llama_index.core.tools import FunctionTool
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.workflow import Context
from llama_index.llms.openai import OpenAI

# Base URL for the FakeStore API
FAKESTORE_API_URL = "https://fakestoreapi.com"

# ---------- Tool Functions ----------

def get_all_products() -> List[Dict]:
    """
    Retrieves all products from the FakeStore API.
    Returns a list of product objects.
    """
    response = requests.get(f"{FAKESTORE_API_URL}/products")
    return response.json()

def get_product_by_id(product_id: int) -> Dict:
    """
    Retrieves a specific product by its ID.

    Args:
        product_id: The ID of the product to retrieve

    Returns:
        A product object
    """
    response = requests.get(f"{FAKESTORE_API_URL}/products/{product_id}")
    return response.json()

def get_products_in_category(category: str) -> List[Dict]:
    """
    Retrieves all products in a specific category.

    Args:
        category: The category name (e.g., 'electronics', 'jewelery', 'men\'s clothing', 'women\'s clothing')

    Returns:
        A list of product objects in the specified category
    """
    response = requests.get(f"{FAKESTORE_API_URL}/products/category/{category}")
    return response.json()

def get_all_categories() -> List[str]:
    """
    Retrieves all available product categories.

    Returns:
        A list of category names
    """
    response = requests.get(f"{FAKESTORE_API_URL}/products/categories")
    return response.json()

def create_product(
    title: str,
    price: float,
    description: str,
    image: str,
    category: str
) -> Dict:
    """
    Creates a new product in the FakeStore API.
    Note: This will not actually add the product to the database, it will simulate a POST request.

    Args:
        title: The title of the product
        price: The price of the product
        description: The product description
        image: URL to the product image
        category: The product category

    Returns:
        The created product object with an ID
    """
    data = {
        "title": title,
        "price": price,
        "description": description,
        "image": image,
        "category": category
    }
    response = requests.post(f"{FAKESTORE_API_URL}/products", json=data)
    return response.json()

def update_product(
    product_id: int,
    data: Dict[str, Any]
) -> Dict:
    """
    Updates an existing product in the FakeStore API.
    Note: This will not actually update the product in the database, it will simulate a PUT request.

    Args:
        product_id: The ID of the product to update
        data: Dictionary containing fields to update (title, price, description, image, and/or category)

    Returns:
        The updated product object
    """
    response = requests.put(f"{FAKESTORE_API_URL}/products/{product_id}", json=data)
    return response.json()

def delete_product(product_id: int) -> Dict:
    """
    Deletes a product from the FakeStore API.
    Note: This will not actually delete the product from the database, it will simulate a DELETE request.

    Args:
        product_id: The ID of the product to delete

    Returns:
        The deleted product object
    """
    response = requests.delete(f"{FAKESTORE_API_URL}/products/{product_id}")
    return response.json()

def limit_products(limit: int, sort: Optional[str] = None) -> List[Dict]:
    """
    Retrieves a limited number of products with optional sorting.

    Args:
        limit: Maximum number of products to return
        sort: Optional sorting direction ('asc' or 'desc')

    Returns:
        A list of product objects
    """
    url = f"{FAKESTORE_API_URL}/products?limit={limit}"
    if sort:
        url += f"&sort={sort}"
    response = requests.get(url)
    return response.json()

def find_products_by_price_range(min_price: float, max_price: float) -> List[Dict]:
    """
    Finds products within a specific price range.

    Args:
        min_price: Minimum price
        max_price: Maximum price

    Returns:
        List of products within the specified price range
    """
    all_products = get_all_products()
    return [p for p in all_products if min_price <= p["price"] <= max_price]

def find_highest_rated_products(count: int = 5) -> List[Dict]:
    """
    Finds the highest rated products.

    Args:
        count: Number of top-rated products to return

    Returns:
        List of the highest rated products
    """
    all_products = get_all_products()
    sorted_products = sorted(all_products, key=lambda p: p["rating"]["rate"], reverse=True)
    return sorted_products[:count]

def get_cart(cart_id: int) -> Dict:
    """
    Retrieves a specific cart by its ID.

    Args:
        cart_id: The ID of the cart to retrieve

    Returns:
        A cart object
    """
    response = requests.get(f"{FAKESTORE_API_URL}/carts/{cart_id}")
    return response.json()

def get_user_cart(user_id: int) -> List[Dict]:
    """
    Retrieves carts for a specific user.

    Args:
        user_id: The ID of the user whose carts to retrieve

    Returns:
        A list of cart objects belonging to the user
    """
    response = requests.get(f"{FAKESTORE_API_URL}/carts/user/{user_id}")
    return response.json()

# ---------- Create Tools ----------

# Create function tools
tools = [
    FunctionTool.from_defaults(get_all_products, name="get_all_products"),
    FunctionTool.from_defaults(get_product_by_id, name="get_product_by_id"),
    FunctionTool.from_defaults(get_products_in_category, name="get_products_in_category"),
    FunctionTool.from_defaults(get_all_categories, name="get_all_categories"),
    FunctionTool.from_defaults(create_product, name="create_product"),
    FunctionTool.from_defaults(update_product, name="update_product"),
    FunctionTool.from_defaults(delete_product, name="delete_product"),
    FunctionTool.from_defaults(limit_products, name="limit_products"),
    FunctionTool.from_defaults(find_products_by_price_range, name="find_products_by_price_range"),
    FunctionTool.from_defaults(find_highest_rated_products, name="find_highest_rated_products"),
    FunctionTool.from_defaults(get_cart, name="get_cart"),
    FunctionTool.from_defaults(get_user_cart, name="get_user_cart"),
]

# Create a tool mapping for easy access
tools_map = {t.metadata.name: t for t in tools}

# Create a vector index over these tools
obj_index = ObjectIndex.from_objects(
    tools,
    index_cls=VectorStoreIndex,
)

# ---------- Setup Agent ----------

def create_agent(api_key, model="gpt-4.1-mini"):
    """
    Creates and returns a FunctionAgent with access to the FakeStore API tools.

    Args:
        api_key: The OpenAI API key
        model: The OpenAI model to use (default: "gpt-4.1-mini")

    Returns:
        A FunctionAgent ready to handle queries
    """
    llm = OpenAI(api_key=api_key, model=model)

    agent = FunctionAgent(
        tool_retriever=obj_index.as_retriever(similarity_top_k=3),
        llm=llm,
        verbose=True,
    )

    return agent

# ---------- Example Usage ----------

async def run_example(agent):
    """
    Runs an example query with the agent.

    Args:
        agent: The initialized agent
    """
    # Create a context to hold the session/state
    ctx = Context(agent)

    # Example queries
    queries = [
        "What categories of products are available?",
        "Show me the top 3 highest rated products",
        "Find me electronics under $200",
        "Get me product details for product ID 1"
    ]

    for query in queries:
        print(f"\n\nQuery: {query}")
        resp = await agent.run(query, ctx=ctx)
        print(f"Response: {str(resp)}")
        print(f"Tool calls: {resp.tool_calls}")

if __name__ == "__main__":
    # Replace with your OpenAI API key
    import os
    import asyncio

    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        print("Please set OPENAI_API_KEY environment variable")
        exit(1)

    agent = create_agent(api_key)
    await run_example(agent)
    # asyncio.run(run_example(agent))



Query: What categories of products are available?
Response: The available product categories are:

1. Electronics
2. Jewelery
3. Men's Clothing
4. Women's Clothing
Tool calls: [ToolCallResult(tool_name='get_all_categories', tool_kwargs={}, tool_id='call_Xp3bedc7R58t992xhVYwO2OI', tool_output=ToolOutput(content='[\'electronics\', \'jewelery\', "men\'s clothing", "women\'s clothing"]', tool_name='get_all_categories', raw_input={'args': (), 'kwargs': {}}, raw_output=['electronics', 'jewelery', "men's clothing", "women's clothing"], is_error=False), return_direct=False)]


Query: Show me the top 3 highest rated products
Response: Here are the top 3 highest rated products:

1. [Silicon Power 256GB SSD 3D NAND A55 SLC Cache Performance Boost SATA III 2.5](https://fakestoreapi.com/img/71kWymZ+c+L._AC_SX679_.jpg)
   - Price: $109
   - Description: 3D NAND flash are applied to deliver high transfer speeds Remarkable transfer speeds that enable faster bootup and improved overall system perform